In [1]:
# !pip install mygene
# !pip install biothings_client

In [5]:
from biothings_client import get_client
mg = get_client("gene")

In [23]:
# キーワードの検索結果件数と一覧

# mg.query(keyword)

# {'took': 12,
#  'total': 2307,
#  'max_score': 394.02466,
#  'hits': [{'_id': '10845',
#    '_score': 394.02466,
#    'entrezgene': '10845',
#    'name': 'caseinolytic mitochondrial matrix peptidase chaperone subunit X',
#    'symbol': 'CLPX',
#    'taxid': 9606},
#   {'_id': '270166',.....

# gene_infoの各項目

In [25]:
gene_info = mg.getgene("ENSG00000141570")

In [9]:
for key,value in gene_info.items():
    print(key,":",type(value))

HGNC : <class 'str'>
MIM : <class 'str'>
_id : <class 'str'>
_version : <class 'int'>
accession : <class 'dict'>
alias : <class 'list'>
ensembl : <class 'dict'>
entrezgene : <class 'str'>
exac : <class 'dict'>
exons : <class 'list'>
exons_hg19 : <class 'list'>
generif : <class 'list'>
genomic_pos : <class 'dict'>
genomic_pos_hg19 : <class 'list'>
go : <class 'dict'>
homologene : <class 'dict'>
interpro : <class 'list'>
ipi : <class 'list'>
map_location : <class 'str'>
name : <class 'str'>
other_names : <class 'list'>
pantherdb : <class 'dict'>
pathway : <class 'dict'>
pdb : <class 'list'>
pfam : <class 'list'>
pharmgkb : <class 'str'>
pharos : <class 'dict'>
prosite : <class 'str'>
reagent : <class 'dict'>
refseq : <class 'dict'>
reporter : <class 'dict'>
retired : <class 'list'>
symbol : <class 'str'>
taxid : <class 'int'>
type_of_gene : <class 'str'>
umls : <class 'dict'>
unigene : <class 'str'>
uniprot : <class 'dict'>
wikipedia : <class 'dict'>


In [30]:
gene_info["ensembl"]
# {'gene': 'ENSG00000141570',
#  'protein': ['ENSP00000269385', 'ENSP00000405058', 'ENSP00000408753'],
#  'transcript': ['ENST00000269385',....],
#  'translation': [{'protein': 'ENSP00000269385', 'rna': 'ENST00000269385'},....],
#  'type_of_gene': 'protein_coding'}
gene_info["ensembl"]["gene"]
# 遺伝子に関する情報
gene_info["ensembl"]["protein"]
# 生成されるタンパク質種類

gene_info["entrezgene"]
#人間とか犬とか生物のカテゴリを表すID

gene_info["pdb"]
#PDB IDは重複しない４文字のコードで、タンパク質
#データバンクの各エントリーに割り振られます。
#最初の文字は必ず1から9までの数字で、残りの３文字は
#アルファベットまたは数字が使われます。

gene_info["generif"]
# 文献と出現テキストのリスト
# [{'pubmed': 17332741,'text': 'CBX8 is an essenti...'}]

gene_info["name"]
#正式名称

gene_info["other_names"]
# 正式名称表記ゆれ

gene_info["pdb"]
#タンパク質固有ID
# ['2N4Q', '3I91', '5EQ0']

['2N4Q', '3I91', '5EQ0']

# データの読み込み

In [1]:
import pandas as pd
data = pd.read_csv('data_summary/PROT/lll_bioc_prot.csv').values.tolist()

In [ ]:
label = ["protein","appearances","search_results"]

In [3]:
data[0:10]

[['ClpX', 6],
 ['ComK', 10],
 ['CsbB', 2],
 ['CtsR', 6],
 ['DnaK', 8],
 ['E sigma E', 6],
 ['EsigmaF', 8],
 ['FlgM', 13],
 ['FtsZ', 25],
 ['GerE', 210]]

In [51]:
keyword = "momo"

In [53]:
mg.query(keyword)

{'took': 4,
 'total': 4,
 'max_score': 13.657011,
 'hits': [{'_id': '114907846',
   '_score': 13.657011,
   'entrezgene': '114907846',
   'name': 'DLA class II histocompatibility antigen, DR-1 beta chain-like',
   'symbol': 'LOC114907846',
   'taxid': 40151},
  {'_id': '30170',
   '_score': 5.9296494,
   'entrezgene': '30170',
   'name': 'dharma',
   'symbol': 'dharma',
   'taxid': 7955},
  {'_id': '282845',
   '_score': 4.838886,
   'entrezgene': '282845',
   'name': 'ring finger protein 34',
   'symbol': 'Rnf34',
   'taxid': 10116},
  {'_id': '80196',
   '_score': 2.1278205,
   'entrezgene': '80196',
   'name': 'ring finger protein 34',
   'symbol': 'RNF34',
   'taxid': 9606}]}

In [94]:
import copy
new_data = copy.deepcopy(data)

In [95]:
new_data[0:15]

[['ClpX', 6],
 ['ComK', 10],
 ['CsbB', 2],
 ['CtsR', 6],
 ['DnaK', 8],
 ['E sigma E', 6],
 ['EsigmaF', 8],
 ['FlgM', 13],
 ['FtsZ', 25],
 ['GerE', 210],
 ['KatX', 8],
 ['KinC', 2],
 ['KinD', 2],
 ['PBP4*', 7],
 ['PhoP~P', 3]]

In [62]:
import urllib

In [ ]:
for i in range(len(new_data)):
    keyword = new_data[i][0]
    try:
        result = mg.query(keyword)
        search_results = result["total"]
        print(i,"/",len(new_data),"   ",keyword,"=>",search_results)
    except:
        print(i,"/",len(new_data),"   HTTPError")
        search_results = -1
    new_data[i].append(search_results)

In [97]:
for i in range(len(new_data[0:10])):
    keyword = new_data[i][0]
    try:
        result = mg.query(keyword)
        search_results = result["total"]
        max_score = result["max_score"]
        
        max_score_hit = []
        if search_results != 0:
            for hit in result["hits"]:
                if hit["_score"] == max_score:
                    max_score_hit.append(hit)
        print(i,"/",len(new_data)-1,"   ",keyword,"=>",search_results,"(max_hits",len(max_score_hit),")")
    except:
        print(i,"/",len(new_data),"   HTTPError")
        search_results = -1
    new_data[i].append(search_results)
    new_data[i].append(len(max_score_hit))

0 / 120     ClpX => 2319 (max_hits 1 )
1 / 120     ComK => 166 (max_hits 3 )
2 / 120     CsbB => 4 (max_hits 3 )
3 / 120     CtsR => 226 (max_hits 1 )
4 / 120     DnaK => 2136 (max_hits 10 )
5 / 120     E sigma E => 1007 (max_hits 10 )
6 / 120     EsigmaF => 0 (max_hits 0 )
7 / 120     FlgM => 514 (max_hits 10 )
8 / 120     FtsZ => 2955 (max_hits 10 )
9 / 120     GerE => 28 (max_hits 10 )


In [68]:
new_data_row_2 = [row[2] for row in new_data]

In [72]:
new_data_row_2

[2319,
 166,
 4,
 226,
 2136,
 1007,
 0,
 514,
 2955,
 28,
 8,
 4,
 212,
 88,
 -1,
 38,
 0,
 17,
 523,
 146,
 11,
 0,
 174,
 -1,
 86,
 0,
 58,
 1079,
 123,
 11,
 124,
 4,
 0,
 0,
 545,
 0,
 2,
 1145,
 20,
 40,
 0,
 539,
 817,
 258,
 3,
 3,
 25,
 2,
 2,
 2,
 514,
 1718,
 28,
 171,
 41,
 8,
 142,
 1117,
 10,
 3,
 220,
 447,
 523,
 146,
 40,
 90,
 1108,
 1047,
 170,
 368,
 278,
 64,
 44,
 5476,
 113,
 142,
 281,
 87,
 26,
 7,
 7,
 1,
 0,
 0,
 2,
 1,
 0,
 0,
 266,
 0,
 0,
 174,
 0,
 0,
 121,
 223,
 2,
 1,
 1,
 12,
 24,
 512,
 458,
 167,
 121,
 36,
 1,
 2,
 2,
 3,
 4,
 1,
 17,
 2,
 1,
 5,
 1,
 0,
 0,
 2,
 0]

In [83]:
keyword = "ykvP"
mg.query(keyword)

{'took': 4,
 'total': 1,
 'max_score': 426.04236,
 'hits': [{'_id': '939278',
   '_score': 426.04236,
   'entrezgene': '939278',
   'name': 'spore protein (HGT island)',
   'symbol': 'ykvP',
   'taxid': 224308}]}

In [74]:
search_results

1

In [93]:
keyword = "YfhP"
mg.query(keyword)

{'took': 6,
 'total': 4,
 'max_score': 413.27167,
 'hits': [{'_id': '936190',
   '_score': 413.27167,
   'entrezgene': '936190',
   'name': 'putative membrane hydrolase',
   'symbol': 'yfhP',
   'taxid': 224308},
  {'_id': '1254066',
   '_score': 413.27167,
   'entrezgene': '1254066',
   'name': 'Fe-S cluster assembly transcriptional regulator IscR',
   'symbol': 'yfhP',
   'taxid': 99287},
  {'_id': '107885630',
   '_score': 54.268234,
   'entrezgene': '107885630',
   'name': 'uncharacterized protein YfhP-like',
   'symbol': 'LOC107885630',
   'taxid': 7029},
  {'_id': '945279',
   '_score': 8.232498,
   'entrezgene': '945279',
   'name': 'DNA-binding transcriptional dual regulator IscR',
   'symbol': 'iscR',
   'taxid': 511145}]}

In [87]:
t